<a href="https://colab.research.google.com/github/vshnu07/resturant_recommendation/blob/main/Resturant_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from math import radians,sin,cos,sqrt,atan2
df_orders = pd.read_csv('/content/orders_cleaned_final.csv')
df_train_customers=pd.read_csv('/content/cleaned_train_customers.csv')
df_train_locations=pd.read_csv('/content/cleaned_train_locations.csv')
df_vendors=pd.read_csv('/content/cleaned_vendors.csv')
df_test_customers=pd.read_csv('/content/cleaned_test_customers.csv')
df_test_locations=pd.read_csv('/content/cleaned_test_locations.csv')

#data type conversion
date_cols_customer=['created_at','updated_at']
for df in [df_train_customers,df_test_customers]:
  for col in date_cols_customer:
    df[col]=pd.to_datetime(df[col],errors='coerce')
date_cols_orders = ['delivery_time', 'order_accepted_time', 'driver_accepted_time','ready_for_pickup_time', 'picked_up_time', 'delivered_time','delivery_date', 'created_at']
for col in date_cols_orders:
    df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')
date_cols_vendors = ['created_at', 'updated_at']
for col in date_cols_vendors:
    df_vendors[col] = pd.to_datetime(df_vendors[col], errors='coerce')


df_orders.info()


/tmp/ipython-input-18-3593205987.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')
/tmp/ipython-input-18-3593205987.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')
/tmp/ipython-input-18-3593205987.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')
/tmp/ipython-input-18-3593205987.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falli

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135288 entries, 0 to 135287
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   order_id                        135288 non-null  int64         
 1   customer_id                     135288 non-null  object        
 2   item_count                      135288 non-null  int64         
 3   grand_total                     135288 non-null  float64       
 4   payment_mode                    135288 non-null  int64         
 5   promo_code                      135288 non-null  object        
 6   vendor_discount_amount          135288 non-null  float64       
 7   promo_code_discount_percentage  135288 non-null  float64       
 8   is_favorite                     135288 non-null  object        
 9   is_rated                        135288 non-null  object        
 10  vendor_rating                   135288 non-null  float64

/tmp/ipython-input-18-3593205987.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')


In [21]:
#feature engineering customer

def create_customer_features(df_customers_input):
  df = df_customers_input.copy()
  max_date = pd.concat([df_train_customers['updated_at'], df_test_customers['updated_at']]).max()
  df['account_age_days'] = (max_date - df['created_at']).dt.days
  df['updated_since_creation_days'] = (df['updated_at'] - df['created_at']).dt.days



  #category of customers
  categorical_cols = ['gender', 'status', 'verified', 'language']
  df[categorical_cols] = df[categorical_cols].astype('category')
  return df


df_train_customers_fe = create_customer_features(df_train_customers)
print("train_customer_features: \n",df_train_customers_fe)
df_test_customers_fe = create_customer_features(df_test_customers)
print("test_customer_features: \n",df_train_customers_fe)




#feature engineering location


def create_location_features(df_locations_input):
    df = df_locations_input.copy()
    customer_location_counts = df.groupby('customer_id')['location_number'].nunique().reset_index()
    customer_location_counts.rename(columns={'location_number': 'num_customer_locations'}, inplace=True)
    df = pd.merge(df, customer_location_counts, on='customer_id', how='left')#left join


    mode_location_type = df.groupby('customer_id')['location_type'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').reset_index()
    mode_location_type.rename(columns={'location_type': 'most_frequent_location_type'}, inplace=True)
    df = pd.merge(df, mode_location_type, on='customer_id', how='left')


    df['location_type'] = df['location_type'].astype('category')
    df['most_frequent_location_type'] = df['most_frequent_location_type'].astype('category')
    return df


df_train_locations_fe = create_location_features(df_train_locations)
df_test_locations_fe = create_location_features(df_test_locations)
print("train_loc\n",df_train_locations_fe)
print("test_loc\n",df_test_locations_fe)





#feature engineering vendors

def create_vendor_features(df_vendors_input):
  df = df_vendors_input.copy()
  max_date_vendors = df['updated_at'].max()
  df['vendor_age_days'] = (max_date_vendors - df['created_at']).dt.days


  #categorey of vendors
  df['vendor_category_en'] = df['vendor_category_en'].astype('category')
  df['is_open'] = df['is_open'].astype('category')
  df['is_haked_delivering'] = df['is_haked_delivering'].astype('category')
  df['status'] = df['status'].astype('category')
  df['verified'] = df['verified'].astype('category')
  df['language'] = df['language'].astype('category')
  df['one_click_vendor'] = df['one_click_vendor'].astype('category')
  df['device_type'] = df['device_type'].astype('category')
  df['display_orders'] = df['display_orders'].astype('category')
  df['open_close_flags'] = df['open_close_flags'].astype('category')



  def parse_time_string(time_str):
    if pd.isna(time_str) or time_str == 'Unknown':
      return np.nan
    try:
      first_time_part = time_str.split('-')[0].strip()
      return pd.to_datetime(first_time_part, errors='coerce').hour
    except:
      return np.nan

  df['opening_hour'] = df['OpeningTime'].apply(parse_time_string)
  df['opening_hour2'] = df['OpeningTime2'].apply(parse_time_string)


  time_columns = [col for col in df.columns if 'from_time' in col or 'to_time' in col]
  for col in time_columns:
    df[col] = df[col].replace('Unknown', np.nan)
  df['num_defined_open_slots'] = df[['sunday_from_time1', 'monday_from_time1', 'tuesday_from_time1','wednesday_from_time1',
                                          'thursday_from_time1', 'friday_from_time1','saturday_from_time1']].notna().sum(axis=1)


  return df

df_vendors_fe = create_vendor_features(df_vendors)

print("\nSample customer features (df_train_customers_fe):")
print(df_train_customers_fe[['customer_id', 'gender', 'account_age_days', 'updated_since_creation_days']].head().to_markdown(index=False, numalign="left", stralign="left"))

print("\nSample vendor features (df_vendors_fe):")
print(df_vendors_fe[['id', 'vendor_category_en', 'vendor_age_days', 'opening_hour', 'num_defined_open_slots']].head().to_markdown(index=False, numalign="left", stralign="left"))


train_customer_features: 
       customer_id   gender status verified language          created_at  \
0         TCHWPBT     Male      1        1       EN 2023-02-07 19:16:00   
1         ZGFSYCZ     Male      1        1       EN 2023-02-09 12:04:00   
2         S2ALZFL     Male      0        1       EN 2023-03-14 18:31:00   
3         952DBJQ     Male      1        1       EN 2023-03-15 19:47:00   
4         1IX6FXS     Male      1        1       EN 2023-03-15 19:57:00   
...           ...      ...    ...      ...      ...                 ...   
34598     JAI79PE  Unknown      1        1  Unknown 2025-03-03 13:01:00   
34599     TR75VFL  Unknown      1        1  Unknown 2025-03-03 13:22:00   
34600     SQMJ08H  Unknown      1        1  Unknown 2025-03-03 13:24:00   
34601     9LW9CHN  Unknown      1        1  Unknown 2025-03-03 13:24:00   
34602     LBMRK7A  Unknown      1        1  Unknown 2025-03-03 13:31:00   

               updated_at  account_age_days  updated_since_creation_days

In [20]:
#feature engineering orders

def create_order_aggregates(df_orders_input):
    df = df_orders_input.copy()


    #agg of customer order history
    customer_order_agg=df.groupby('customer_id').agg(
        avg_item_count=('item_count', 'mean'),
        avg_grand_total=('grand_total', 'mean'),
        total_orders=('order_id', 'nunique'),
        most_frequent_payment_mode=('payment_mode', lambda x: x.mode()[0] if not x.mode().empty else -1),
        fav_orders_ratio=('is_favorite', lambda x: (x == 'Yes').mean() if x.dtype == 'object' else x.mean()),
        rated_orders_ratio=('is_rated', lambda x: (x == 'Yes').mean() if x.dtype == 'object' else x.mean()),
        avg_customer_vendor_rating=('vendor_rating', 'mean'),
        avg_customer_driver_rating=('driver_rating', 'mean'),
        avg_delivery_distance=('deliverydistance', 'mean'),
        avg_preparation_time=('preparationtime', 'mean'),
        time_since_last_order_days=('created_at', lambda x: (df['created_at'].max() - x.max()).days)).reset_index()




    # Aggregate vendor order history
    vendor_order_agg = df.groupby('vendor_id').agg(
        vendor_total_orders=('order_id', 'nunique'),
        vendor_avg_item_count=('item_count', 'mean'),
        vendor_avg_grand_total=('grand_total', 'mean'),
        vendor_avg_delivery_distance=('deliverydistance', 'mean'),
        vendor_avg_preparation_time=('preparationtime', 'mean'),
        vendor_promo_code_usage_rate=('promo_code', lambda x: (x != 'No Promo').mean())).reset_index()

    return customer_order_agg, vendor_order_agg

df_customer_order_agg, df_vendor_order_agg = create_order_aggregates(df_orders)
print("\nagg customer and vendor")



#distance calculation
def haversine(lat1, lon1, lat2, lon2):  #circle distance
  R = 6371  #radius of earth in km
  lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance


agg customer and vendor


In [26]:
#Training Data(Positive and Negative Samples)

# Standardize column name for location number in df_orders
df_orders.rename(columns={'LOCATION_NUMBER': 'location_number',
                          'LOCATION_TYPE': 'location_type'}, inplace=True)

df_train_positive = df_orders[['customer_id', 'location_number', 'vendor_id']].copy()
df_train_positive['ordered'] = 1
print(f"\nPositive samples (from orders): {len(df_train_positive)}")

df_customer_loc_coords = df_train_locations.copy()
df_customer_loc_coords = df_customer_loc_coords[['customer_id', 'location_number', 'latitude', 'longitude']]
df_customer_loc_coords.rename(columns={'latitude': 'customer_lat', 'longitude': 'customer_lon'}, inplace=True)

df_vendor_coords = df_vendors[['id', 'latitude', 'longitude', 'serving_distance']].copy()
df_vendor_coords.rename(columns={'id': 'vendor_id', 'latitude': 'vendor_lat', 'longitude': 'vendor_lon'}, inplace=True)

all_possible_interactions = []
unique_train_customer_locations = df_orders[['customer_id', 'location_number']].drop_duplicates()

unique_train_customer_locations = pd.merge(
    unique_train_customer_locations,
    df_customer_loc_coords,
    on=['customer_id', 'location_number'],
    how='left'
)

for _, row_cust_loc in unique_train_customer_locations.iterrows():
    customer_id = row_cust_loc['customer_id']
    location_number = row_cust_loc['location_number']
    customer_lat = row_cust_loc['customer_lat']
    customer_lon = row_cust_loc['customer_lon']

    if pd.notna(customer_lat) and pd.notna(customer_lon):
        distances = df_vendor_coords.apply(
            lambda x: haversine(customer_lat, customer_lon, x['vendor_lat'], x['vendor_lon']),
            axis=1
        )
        eligible_vendors = df_vendor_coords[
            (distances <= df_vendor_coords['serving_distance']) | (df_vendor_coords['serving_distance'] == 0)
        ]
    else:
        eligible_vendors = df_vendor_coords.copy()

    for vendor_id in eligible_vendors['vendor_id'].unique():
        all_possible_interactions.append({
            'customer_id': customer_id,
            'location_number': location_number,
            'vendor_id': vendor_id
        })

df_all_possible_interactions = pd.DataFrame(all_possible_interactions)
print(f"Total possible customer-location-vendor interactions (after distance filter): {len(df_all_possible_interactions)}")

df_all_possible_interactions['ordered'] = 0
df_all_possible_interactions = df_all_possible_interactions.merge(
    df_train_positive,
    on=['customer_id', 'location_number', 'vendor_id'],
    how='left',
    suffixes=('_potential', '_actual')
)
df_all_possible_interactions['ordered'] = df_all_possible_interactions['ordered_actual'].fillna(0).astype(int)
df_all_possible_interactions.drop(columns=['ordered_potential', 'ordered_actual'], inplace=True)

negatives_pool = df_all_possible_interactions[df_all_possible_interactions['ordered'] == 0]
sample_size = min(len(negatives_pool), len(df_train_positive) * 5)
df_train_negative_sampled = negatives_pool.sample(n=sample_size, random_state=42)

df_train_data = pd.concat([df_train_positive, df_train_negative_sampled]).drop_duplicates().reset_index(drop=True)
print(f"Final training data size (positives + sampled negatives): {len(df_train_data)}")

# Merge all features into the training data
df_train_data = pd.merge(df_train_data, df_train_customers_fe, on='customer_id', how='left')
df_train_data.drop(columns=['created_at', 'updated_at'], inplace=True)

df_train_data = pd.merge(df_train_data, df_train_locations_fe, on=['customer_id', 'location_number'], how='left')
df_train_data.rename(columns={'latitude': 'customer_loc_lat', 'longitude': 'customer_loc_lon'}, inplace=True)

df_train_data = pd.merge(df_train_data, df_vendors_fe, left_on='vendor_id', right_on='id', how='left')
df_train_data.drop(columns=['id', 'created_at_y', 'updated_at_y'], inplace=True,errors='ignore')
df_train_data.rename(columns={'latitude': 'vendor_lat_val', 'longitude': 'vendor_lon_val',
                              'created_at_x': 'customer_created_at'}, inplace=True)

df_train_data = pd.merge(df_train_data, df_customer_order_agg, on='customer_id', how='left')
df_train_data = pd.merge(df_train_data, df_vendor_order_agg, on='vendor_id', how='left')

df_train_data['customer_vendor_distance'] = df_train_data.apply(
    lambda row: haversine(row['customer_loc_lat'], row['customer_loc_lon'],
                          row['vendor_lat_val'], row['vendor_lon_val'])
    if pd.notna(row['customer_loc_lat']) and pd.notna(row['vendor_lat_val']) else np.nan,
    axis=1
)

print("\nAll features merged into training data.")
print(f"Final training data shape: {df_train_data.shape}")


Positive samples (from orders): 135288
Total possible customer-location-vendor interactions (after distance filter): 99961
Final training data size (positives + sampled negatives): 174326

All features merged into training data.
Final training data shape: (174596, 95)


In [27]:
#Model Training

target = 'ordered'
exclude_cols = [
    'customer_id', 'location_number', 'vendor_id', 'customer_created_at',
    'OpeningTime', 'OpeningTime2',
    'sunday_from_time1', 'sunday_to_time1', 'sunday_from_time2', 'sunday_to_time2',
    'monday_from_time1', 'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
    'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2', 'tuesday_to_time2',
    'wednesday_from_time1', 'wednesday_to_time1', 'wednesday_from_time2', 'wednesday_to_time2',
    'thursday_from_time1', 'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
    'friday_from_time1', 'friday_to_time1', 'friday_from_time2', 'friday_to_time2',
    'saturday_from_time1', 'saturday_to_time1', 'saturday_from_time2', 'saturday_to_time2',
    'customer_loc_lat', 'customer_loc_lon', 'vendor_lat_val', 'vendor_lon_val',
    'vendor_tag', 'vendor_tag_name', 'primary_tags',
    'created_at', 'updated_at'
]

features = [col for col in df_train_data.columns if col not in exclude_cols + [target]]

X = df_train_data[features]
y = df_train_data[target]

categorical_features = X.select_dtypes(include='category').columns.tolist()
for col in X.select_dtypes(include='object').columns:
    if col not in categorical_features:
        X[col] = X[col].astype('category')
        categorical_features.append(col)

print(f"\nFeatures selected ({len(features)}): {features}")
print(f"Categorical features identified ({len(categorical_features)}): {categorical_features}")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nData split into training ({len(X_train)} samples) and validation ({len(X_val)} samples).")

lgb_clf = lgb.LGBMClassifier(objective='binary', metric='auc', random_state=42)
lgb_clf.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='auc',
            callbacks=[lgb.early_stopping(10, verbose=False)],
            categorical_feature=categorical_features
           )

print("\nLightGBM model trained successfully.")
print(f"Validation AUC: {lgb_clf.best_score_['valid_0']['auc']:.4f}")


Features selected (52): ['gender', 'status_x', 'verified_x', 'language_x', 'account_age_days', 'updated_since_creation_days', 'location_type', 'num_customer_locations', 'most_frequent_location_type', 'authentication_id', 'vendor_category_en', 'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'prepration_time', 'commission', 'is_haked_delivering', 'discount_percentage', 'status_y', 'verified_y', 'rank', 'language_y', 'vendor_rating', 'open_close_flags', 'one_click_vendor', 'country_id', 'city_id', 'device_type', 'display_orders', 'vendor_age_days', 'opening_hour', 'opening_hour2', 'num_defined_open_slots', 'avg_item_count', 'avg_grand_total', 'total_orders', 'most_frequent_payment_mode', 'fav_orders_ratio', 'rated_orders_ratio', 'avg_customer_vendor_rating', 'avg_customer_driver_rating', 'avg_delivery_distance', 'avg_preparation_time', 'time_since_last_order_days', 'vendor_total_orders', 'vendor_avg_item_count', 'vendor_avg_grand_total', 'vendor_avg_delivery_dista

In [30]:
#Prediction on Test Set and Submission

print("\nPreparing test data for prediction...")

df_test_locations.rename(columns={'LOCATION_NUMBER': 'location_number',
                                  'LOCATION_TYPE': 'location_type'}, inplace=True)

df_test_customer_locations = df_test_locations[['customer_id', 'location_number']].drop_duplicates().copy()

df_test_customer_loc_coords = df_test_locations.copy()
df_test_customer_loc_coords = df_test_customer_loc_coords[['customer_id', 'location_number', 'latitude', 'longitude']]
df_test_customer_loc_coords.rename(columns={'latitude': 'customer_lat', 'longitude': 'customer_lon'}, inplace=True)

df_test_candidates = []
for _, row_cust_loc in df_test_customer_locations.iterrows():
    customer_id = row_cust_loc['customer_id']
    location_number = row_cust_loc['location_number']

    customer_coords_row = df_test_customer_loc_coords[
        (df_test_customer_loc_coords['customer_id'] == customer_id) &
        (df_test_customer_loc_coords['location_number'] == location_number)
    ]
    if not customer_coords_row.empty:
        customer_lat = customer_coords_row['customer_lat'].iloc[0]
        customer_lon = customer_coords_row['customer_lon'].iloc[0]
    else:
        customer_lat = np.nan
        customer_lon = np.nan

    if pd.notna(customer_lat) and pd.notna(customer_lon):
        distances = df_vendor_coords.apply(
            lambda x: haversine(customer_lat, customer_lon, x['vendor_lat'], x['vendor_lon']),
            axis=1
        )
        eligible_vendors = df_vendor_coords[
            (distances <= df_vendor_coords['serving_distance']) | (df_vendor_coords['serving_distance'] == 0)
        ]
    else:
        eligible_vendors = df_vendor_coords.copy()

    for vendor_id in eligible_vendors['vendor_id'].unique():
        df_test_candidates.append({
            'customer_id': customer_id,
            'location_number': location_number,
            'vendor_id': vendor_id
        })

df_test_candidates = pd.DataFrame(df_test_candidates)
print(f"Total test candidate customer-location-vendor interactions: {len(df_test_candidates)}")

df_test_data = pd.merge(df_test_candidates, df_test_customers_fe, on='customer_id', how='left')
df_test_data.drop(columns=['created_at', 'updated_at'], inplace=True)

df_test_data = pd.merge(df_test_data, df_test_locations_fe, on=['customer_id', 'location_number'], how='left')
df_test_data.rename(columns={'latitude': 'customer_loc_lat', 'longitude': 'customer_loc_lon'}, inplace=True)

df_test_data = pd.merge(df_test_data, df_vendors_fe, left_on='vendor_id', right_on='id', how='left')
df_test_data.drop(columns=[col for col in ['id', 'created_at_y', 'updated_at_y'] if col in df_test_data.columns], inplace=True)
df_test_data.rename(columns={'latitude': 'vendor_lat_val', 'longitude': 'vendor_lon_val',
                             'created_at_x': 'customer_created_at'}, inplace=True)

df_test_data = pd.merge(df_test_data, df_customer_order_agg, on='customer_id', how='left')
df_test_data = pd.merge(df_test_data, df_vendor_order_agg, on='vendor_id', how='left')

df_test_data['customer_vendor_distance'] = df_test_data.apply(
    lambda row: haversine(row['customer_loc_lat'], row['customer_loc_lon'],
                          row['vendor_lat_val'], row['vendor_lon_val'])
    if pd.notna(row['customer_loc_lat']) and pd.notna(row['vendor_lat_val']) else np.nan,
    axis=1
)

for col in X.columns:
    if col in df_test_data.columns:
        if df_test_data[col].dtype in ['float64', 'int64']:
            df_test_data[col].fillna(0, inplace=True)
        elif df_test_data[col].dtype.name == 'category' or df_test_data[col].dtype == 'object':
            if 'Unknown' not in df_test_data[col].cat.categories:
              df_test_data[col] = df_test_data[col].cat.add_categories(['Unknown'])
              df_test_data[col].fillna('Unknown', inplace=True)

            df_test_data[col] = df_test_data[col].astype('category')
            if not pd.api.types.is_categorical_dtype(df_test_data[col]):
                df_test_data[col] = df_test_data[col].astype('category')
            train_categories = X_train[col].cat.categories
            test_categories = df_test_data[col].cat.categories
            missing_in_test = train_categories.difference(test_categories)
            if len(missing_in_test) > 0:
                df_test_data[col] = df_test_data[col].cat.add_categories(list(missing_in_test))

X_test_final = df_test_data[X_train.columns]

print("\nPredicting probabilities for test set...")
test_probabilities = lgb_clf.predict_proba(X_test_final)[:, 1]
df_test_data['predicted_probability'] = test_probabilities



Preparing test data for prediction...
Total test candidate customer-location-vendor interactions: 35721


/tmp/ipython-input-30-984169402.py:82: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_test_data[col]):
/tmp/ipython-input-30-984169402.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test_data[col].fillna('Unknown', inplace=True)
/tmp/ipython-input-30-984169402.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment usin


Predicting probabilities for test set...


In [31]:
df_submission = df_test_data.copy()
df_submission['rank'] = df_submission.groupby(['customer_id', 'location_number'])['predicted_probability'].rank(ascending=False, method='first')

top_n = 5
df_submission_top_n = df_submission[df_submission['rank'] <= top_n].copy()

df_submission_top_n['CID X LOC_NUM X VENDOR'] = df_submission_top_n['customer_id'] + ' X ' + \
                                                  df_submission_top_n['location_number'].astype(str) + ' X ' + \
                                                  df_submission_top_n['vendor_id'].astype(str)

df_submission_top_n = df_submission_top_n.sort_values(
    by=['customer_id', 'location_number', 'rank']
)

df_final_submission = df_submission_top_n[['CID X LOC_NUM X VENDOR']]

submission_filename = 'recommendation_submission.csv'
df_final_submission.to_csv(submission_filename, index=False)

print(f"\nRecommendation engine built and submission file '{submission_filename}' generated successfully.")
print(f"Submission file head:\n{df_final_submission.head().to_markdown(index=False, numalign='left', stralign='left')}")


Recommendation engine built and submission file 'recommendation_submission.csv' generated successfully.
Submission file head:
| CID X LOC_NUM X VENDOR   |
|:-------------------------|
| 01LAENG X 1 X 43         |
| 01LAENG X 1 X 191        |
| 01LAENG X 1 X 134        |
| 01LAENG X 1 X 676        |
| 01LAENG X 2 X 201        |
